<a href="https://colab.research.google.com/github/donghuna/AI-Expert/blob/main/%EC%96%91%EC%9D%B8%EC%88%9C/tabular_mdp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/day1_tabular_mdp
%ls

/content/drive/MyDrive/day1_tabular_mdp
chap0_gym_intro.pdf    gridworld.py     pendulum.py   tabular_mdp_full.ipynb
chap1_tabular_mdp.pdf  gym_intro.ipynb  __pycache__/  tabular_mdp.ipynb
gridworld.png          gym_test.py      solvers.py    utils.py


In [3]:
import numpy as np
import time
from utils import *

# 1. Prerequisites

## 1.1 Toy Example

In [4]:
R = np.array([[-2.0, -0.5],
              [-1.0, -3.0]])

P = np.array([[0.75, 0.25],
              [0.75, 0.25],
              [0.25, 0.75],
              [0.25, 0.75]])

gamma = 0.9

## 1.2 Define Q-function and Greedy policy
First define Q-function from the value function:
$$Q(s, a) = r(s, a) + \gamma \sum_{s^\prime} p(s^\prime \mid s, a) v(s^\prime)$$

Greedy policy from Q-function: $\pi_{\text{greedy}}(s) = \arg\max_a Q(s, a)$.

In [11]:
def q_ftn(P, R, gamma, v):
    """
    given v, get corresponding q
    hint : v's shape = (|S|, 1)
           P's shape = (|S| * |A|, |S|)
           R's shape = (|S|, |A|)
           [Fill here]'s shape = (|S| * |A|, 1)
           numpy.matmul(A, x) returns a matrix multiplication result A * x.
    S 상태에서 A action을 했을때 받을 수 있는 최대 누적 reward 값

    """
    # TODO 1: Complete the following line.
    # return R + gamma * np.matmul(P, v)
    return R + gamma * np.reshape(P@v, newshape=R.shape, order='F')

def greedy(P, R, gamma, v):
    """
    construct greedy policy by pi(s) = argmax_a q(s, a)
    """
    # TODO 2: Complete the following line.
    q = q_ftn(P, R, gamma, v)
    pi = np.argmax(q, axis=1)

    return pi

# 2. Value Iteration

Value iteration iteratively applies the *Bellman Optimality Operator*

\begin{equation*}
\mathcal{T}v\,(s) := \max_{a \in \mathcal{A}}\bigg[ r\,(s,a) + \gamma \sum_{s'\in\mathcal{S}} p\,(s'|s,a)\;v\,(s') \bigg]
\end{equation*}

From calculated $v^*$, we can calculate $\pi^*$ by Q-function

$$ \pi^* := \arg\max_{a\in\mathcal{A}} Q^*(s,a) $$

In [8]:
def bellman_update(P, R, gamma, v):
    """
    implementation of one-step Bellman update
    return : vector of shape (|S|, 1) which corresponds to Tv, where T is Bellman operator
    """

    q = q_ftn(P, R, gamma, v)
    v_next = np.max(q, axis=1, keepdims=True)   # computation of Bellman operator Tv

    return v_next

In [9]:
def print_result(v, pi, mode='discount'):
    print('+========== Result ==========+')
    nS = v.size
    S = range(nS)

    print('optimal value function : ')
    # print a given value function
    for s in S:
        print('v(s{}) = {}'.format(s + 1, v[s, 0]))

    print('optimal policy : ')
    # print a given policy
    for s in S:
        print('pi(s{}) = a{}'.format(s + 1, pi[s] + 1))
    return

In [13]:
EPS = 1e-6
nS, nA = R.shape
# Initialize v
v = np.zeros(shape=(nS, 1), dtype=float)

# Value Iteration
count = 0
start = time.time()
while True:
    v_next = bellman_update(P, R, gamma, v)
    # TODO 3: Fill the blank below.
    if np.linalg.norm(v_next - v, ord=np.inf) < EPS:
        break
    v = v_next
    count += 1
print('Iteration terminated in {} steps, Ellapsed time {:4f} sec\n'.format(count, time.time() - start))

# TODO 4: Construct policy obtained v^* .
pi = greedy(P, R, gamma, v)
print_result(v, pi)

Iteration terminated in 129 steps, Ellapsed time 0.008169 sec

+========== Result ==========+
optimal value function : 
v(s1) = -7.327576823826019
v(s2) = -7.672404410032915
optimal policy : 
pi(s1) = a2
pi(s2) = a1


+========== Result ==========+  
optimal value function :   
v(s1) = -7.327576823826019 // s1에 있을 때의 기대치  
v(s2) = -7.672404410032915  
s1에 있는 것이 좀 더 좋음을 의미  
optimal policy :   
pi(s1) = a2 // S1에 있다면 A2를 취해라  
pi(s2) = a1 // S2에 있다면 A1을 취해라  


# 2. Policy Iteration

Policy Iteration iterates between these 2 steps


*   Policy Evaluation : calculate policy value function $v^\pi$
*   Policy Update : from calculated $v^\pi$, update $\pi$

## 2.1 Policy Evaluation: Get induced dynamics

Let, start from getting $\hat{P}$ and $\hat{R}$

Suppose 2 states, 2 actions, and $\pi = [a_1, a_0]$

We should select row vector of $P$
\begin{equation*}
p(\cdot|s_0,a_0)\\
p(\cdot|s_1,a_0) \rightarrow p(\cdot|s_1,\pi(s_1))\\
p(\cdot|s_0,a_1) \rightarrow p(\cdot|s_0,\pi(s_0))\\
p(\cdot|s_1,a_1)\\
\end{equation*}

So, $P^\pi$ is a rearranged submatrix of $P$

$$P^{\pi} = P\;[\;[2,1], \; : \;]$$

Note that row number can be obtained via:

$$2 = 0(s_0) + 2 \cdot 1(\pi(s_0))$$

$$1 = 1(s_1) + 2 \cdot 0(\pi(s_1))$$

In [ ]:
def induced_dynamic(nS, P, R, pi):
    """
    given policy pi, compute induced dynamic P^pi & R^pi
    """
    # TODO 5: Compute 'row number' with formulations above.
    rows =

    # TODO 6: Get P_pi and R_pi from P and R.
    P_pi =
    R_pi = np.array([["""blank"""]] for s in range(nS)])

    return P_pi, R_pi

## 2.2 Policy Evaluation: Calculate $v^\pi$
With calculated induced dynamics, we can compute $v^\pi$


\begin{equation*}
v^\pi = (I - \gamma P^\pi)^{-1} R^{\pi}
\end{equation*}


To solve the above equation, you may use ```numpy.linalg.solve(A, b)```.
This returns a solution of the linear equations $A x = b$, i.e., $x = A^{-1}b$.

Note that $v^\pi$ can be attained with iterating the Bellman operator $\mathcal{T}^\pi$

$$\mathcal{T}^\pi v := R^\pi +\gamma P^\pi v^\pi $$

In [ ]:
def eval_policy(nS, P, R, gamma, pi):
    """
    policy evaluation
    """
    P_pi, R_pi = induced_dynamic(nS, P, R, pi)

    Id = np.identity(nS)
    # discounted reward problem
    A = Id - gamma * P_pi
    b = R_pi
    v_pi = np.linalg.solve(A, b)

    return v_pi

## 2.2 Policy Update

Based on Q-function from $v^\pi$, let's update our policy

\begin{equation*}
\pi'(s) = \arg\max_{a \in \mathcal{A}} Q^\pi(s,a)
\end{equation*}

## 2.3 Policy Iteration Loop

In [ ]:
nS, nA = R.shape

# initialize policy
pi = np.random.randint(nA, size=nS)

count = 0
start = time.time()
while True:
    v = eval_policy(nS, P, R, gamma, pi)
    # TODO 7: Update your policy using v.
    pi_next =
    if (pi_next == """blank""").all():
        break
    pi =
    count += 1
print('Iteration terminated in {} steps, Ellapsed time {:4f} sec\n'.format(count, time.time() - start))

print_result(v, pi)

## Example: Gridworld

In [ ]:
from gridworld import GridWorld, plot_heatmap

P = GridWorld.P
R = GridWorld.R

gamma = 0.9

In [ ]:
from utils import *
import time

**Value Iteration**

In [ ]:
EPS = 1e-6
nS, nA = R.shape
# initialize v
v = np.zeros(shape=(nS, 1), dtype=float)

begin = time.time()
count = 0
while True:
    v_next = bellman_update(P, R, gamma, v)
    if np.linalg.norm(v_next - v, ord=np.inf) < EPS:
        break
    v = v_next
    count += 1
pi = greedy(P, R, gamma, v)
v_vi = v
print('Value iteration terminated in {} steps.'.format(count))
print('Elapsed time = ', time.time() - begin, 'sec')

**Policy Iteration**

In [ ]:
nS, nA = R.shape

# initialize policy
pi = np.random.randint(nA, size=nS)

begin = time.time()
count = 0
while True:
    v = eval_policy(nS, P, R, gamma, pi)
    pi_next = greedy(P, R, gamma, v)
    if (pi_next == pi).all():
        break
    pi = pi_next
    count += 1
print('Policy iteration terminated in {} steps.'.format(count))
v_pi = v
print('Elapsed time = ', time.time() - begin, 'sec')

In [ ]:
plot_heatmap(v_vi, v_pi)